In [58]:
#Idea: Prediction of groundwater levels: I came across a research paper titled "Groundwater Prediction Using Machine-Learning Tools" (https://www.mdpi.com/1999-4893/13/11/300) and thought of comparing results from regression analysis, SVM and random forests to predict the groundwater levels in India. I use values of precipitation (pr), altitude (alt), average annual temperature (at) and distance from sea (dsea) to predict the groundwater levels.  

#Progress: Managed to run basic GLM and SVM models - I need to undersand the math better to improve them.

In [59]:
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

def clear_dataframe():
    return pd.DataFrame()

In [60]:
#Converting the dta files from American Economic Association Journal's Appendix (paper titled "Wells, Water, and Welfare: The Impact of Access to Groundwater on Rural Poverty and Conflict")

iac = pd.io.stata.read_stata(r'C:\Users\vishe\Desktop\Groundwater\AEJ_IAC.dta')
iac.to_csv(r'C:\Users\vishe\Desktop\Groundwater\AEJ_IAC.csv')

In [61]:
import pandas as pd

def aquifer_depth_str(dmaq2, dmaq3):
    if dmaq3 == 1:
        return 'high'
    elif dmaq2 == 1:
        return 'medium'
    else:
        return 'low'

# Create a new column 'aquifer_depth_str'
iac['aquifer_depth_str'] = iac.apply(lambda row: aquifer_depth_str(row['dmaq2'], row['dmaq3']), axis=1)

def aquifer_depth(dmaq2, dmaq3):
    if dmaq3 == 1:
        return 3
    elif dmaq2 == 1:
        return 2
    else:
        return 1

# Create a new column 'aquifer_depth'
iac['aquifer_depth'] = iac.apply(lambda row: aquifer_depth(row['dmaq2'], row['dmaq3']), axis=1)

columns_to_concat = ['alt', 'dsea', 'at', 'pr', 'aquifer_depth_str', 'aquifer_depth']
iac = pd.concat([iac[columns_to_concat]], axis=1)


In [62]:
iac.head

<bound method NDFrame.head of       alt       dsea         at          pr aquifer_depth_str  aquifer_depth
0     168  29.100000  27.790001  102.660004              high              3
1     168  29.100000  28.059999   76.370003              high              3
2     168  29.100000  28.420000   91.559998              high              3
3     168  29.100000  28.090000   87.480003              high              3
4     168  29.100000  28.129999   69.320000              high              3
...   ...        ...        ...         ...               ...            ...
3963  200  96.800003  25.950001  114.419998               low              1
3964  200  96.800003  25.940001  145.750000               low              1
3965  200  96.800003  26.389999   95.639999               low              1
3966  200  96.800003  26.219999  121.589996               low              1
3967  200  96.800003  26.750000  116.029999               low              1

[3968 rows x 6 columns]>

In [63]:
#Method 2: SVM

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Assuming 'iac' is your DataFrame
X = iac[['alt','dsea', 'at', 'pr']]  # Include your features
y = iac['aquifer_depth_str']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=4)

# Print shapes of training and testing sets for debugging
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Create an SVM classifier
svm_classifier = SVC(kernel='linear', C=1)

# Train the classifier on the training set
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
#accuracy = accuracy_score(y_test, y_pred)
#print(f"Accuracy: {accuracy}")

print("accuarcy score is : ", accuracy_score(y_test, y_pred))


X_train shape: (2777, 4)
y_train shape: (2777,)
X_test shape: (1191, 4)
y_test shape: (1191,)
accuarcy score is :  0.6120906801007556


In [64]:
#Method 1: Generalised Linear Model

model = smf.glm(formula = "aquifer_depth ~ alt + dsea + at + pr", 
                data = iac, 
                family = sm.families.Binomial())

# Fit the model
result = model.fit()
# Display and interpret results
print(result.summary())

#Check error


                 Generalized Linear Model Regression Results                  
Dep. Variable:          aquifer_depth   No. Observations:                 3968
Model:                            GLM   Df Residuals:                     3963
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                   -inf
Date:                Thu, 28 Dec 2023   Deviance:                   2.7862e+05
Time:                        00:34:39   Pearson chi2:                 2.31e+19
No. Iterations:                     5   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -3.012e+16   4.17e+07  -7.22e+08      0.0

C:\Users\vishe\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\genmod\families\links.py:198: RuntimeWarning: overflow encountered in exp
  t = np.exp(-z)
C:\Users\vishe\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\genmod\families\family.py:1056: RuntimeWarning: invalid value encountered in log
  special.gammaln(n - y + 1) + y * np.log(mu / (1 - mu + 1e-20)) +
C:\Users\vishe\AppData\Local\Programs\Python\Python39\lib\site-packages\statsmodels\genmod\families\family.py:1057: RuntimeWarning: invalid value encountered in log
  n * np.log(1 - mu + 1e-20)) * var_weights
